# CS480: Database Systems, Group Project
### Green Taxi Datasets

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import sqlite3
from hopcroftkarp import HopcroftKarp
import time

#### 1. First, we will import the data in the system

In [2]:
green = pd.read_csv('green_tripdata_2015_6months_cleaned.csv')

#### 2. Now, we will get the summary of the datasets and then we will clean up the dataset

In [3]:
green.describe()
# print(green.dtypes)

,trip_id,VendorID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance
count,5.664142e+06,5.664142e+06,5.664142e+06,5.664142e+06,5.664142e+06,5.664142e+06,5.664142e+06,5.664142e+06
mean,3.147373e+06,1.768861e+00,-7.387107e+01,4.071609e+01,-7.387492e+01,4.071671e+01,1.080261e+00,2.888976e+00
std,1.815500e+06,4.215612e-01,2.175846e+00,1.199889e+00,2.093007e+00,1.154116e+00,2.727397e-01,3.000718e+00
min,0.000000e+00,1.000000e+00,-1.151770e+02,0.000000e+00,-1.151766e+02,0.000000e+00,0.000000e+00,1.000000e-02
25%,1.575777e+06,2.000000e+00,-7.395882e+01,4.070032e+01,-7.396673e+01,4.070225e+01,1.000000e+00,1.090000e+00
50%,3.146548e+06,2.000000e+00,-7.394486e+01,4.074728e+01,-7.394425e+01,4.074946e+01,1.000000e+00,1.910000e+00
75%,4.719703e+06,2.000000e+00,-7.391721e+01,4.080458e+01,-7.390993e+01,4.079577e+01,1.000000e+00,3.650000e+00
max,6.291449e+06,2.000000e+00,0.000000e+00,4.269681e+01,0.000000e+00,4.509433e+01,2.000000e+00,8.304000e+02


In [4]:
# Getting the values which needs to be cleaned up before procedding further
print("\u0332".join('Number of Null data values in each columns:'))
print(green.isnull().sum())
print('')
print("\u0332".join('Number of Datasets:'), len(green.index))

N̲u̲m̲b̲e̲r̲ ̲o̲f̲ ̲N̲u̲l̲l̲ ̲d̲a̲t̲a̲ ̲v̲a̲l̲u̲e̲s̲ ̲i̲n̲ ̲e̲a̲c̲h̲ ̲c̲o̲l̲u̲m̲n̲s̲:
trip_id                  0
VendorID                 0
lpep_pickup_datetime     0
Lpep_dropoff_datetime    0
Pickup_longitude         0
Pickup_latitude          0
Dropoff_longitude        0
Dropoff_latitude         0
Passenger_count          0
Trip_distance            0
dtype: int64

N̲u̲m̲b̲e̲r̲ ̲o̲f̲ ̲D̲a̲t̲a̲s̲e̲t̲s̲: 5664142


In [5]:
# Since the number of null values in the 'Ehail_fee' column is equal to the number of rows in the dataset
# Therefore, we will drop the whole column due to its irrelevance.

# green = green.drop(columns=['Ehail_fee'])
# print("\u0332".join('Number of Null data values in each columns:'))
# print(green.isnull().sum())

In [6]:
# Now we will boxplot the 'Trip_type' column
# boxplot = green.boxplot(column=['Trip_type '])

In [7]:
# In the boxplot, we can see that '2.0' is an outlier, therefore majority of the dataset have value '1'
# So, we will replace the null values with '1'
# green['Trip_type '] = green['Trip_type '].fillna(1)

In [8]:
# Now there are no null values in the dataset
# Update the columns variable according to the new cleaned up value
# columns = green.columns
# print("\u0332".join('Number of Null data values in each columns:'))
# print(green.isnull().sum())

#### 3. Now, filter out the data and clean it up again and to get the data we need use for our algorithms

In [9]:
dataset = green[['VendorID', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime', 'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude', 'Passenger_count', 'Trip_distance']]

# Here we are storing all the column names in a array named 'columns'
columns = dataset.columns
print("\u0332".join('Column Names:'), columns)

print("\u0332".join('Dataset:'), len(dataset.index))
dataset = dataset[ (dataset.lpep_pickup_datetime != dataset.Lpep_dropoff_datetime) & (dataset.Trip_distance != 0) & (dataset.Passenger_count < 3) & (dataset['Trip_distance'] != dataset['Trip_distance'].max())]

print("\u0332".join('Filtered Dataset:'), len(dataset.index))

C̲o̲l̲u̲m̲n̲ ̲N̲a̲m̲e̲s̲: Index(['VendorID', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime',
       'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude',
       'Dropoff_latitude', 'Passenger_count', 'Trip_distance'],
      dtype='object')
D̲a̲t̲a̲s̲e̲t̲: 5664142
F̲i̲l̲t̲e̲r̲e̲d̲ ̲D̲a̲t̲a̲s̲e̲t̲: 5638470


In [10]:
print("\u0332".join('Total Distance Travelled:'), dataset['Trip_distance'].sum())

T̲o̲t̲a̲l̲ ̲D̲i̲s̲t̲a̲n̲c̲e̲ ̲T̲r̲a̲v̲e̲l̲l̲e̲d̲: 16332776.31000001


In [11]:
# Correcting the date and time format
dataset['lpep_pickup_datetime'] = pd.to_datetime(dataset['lpep_pickup_datetime'])
dataset['Lpep_dropoff_datetime'] = pd.to_datetime(dataset['Lpep_dropoff_datetime'])
dataset.head()

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance
0,2,2015-01-01 00:34:00,2015-01-01 00:38:00,-73.922592,40.754528,-73.913635,40.765522,1,0.88
1,2,2015-01-01 00:34:00,2015-01-01 00:47:00,-73.952751,40.677711,-73.981529,40.658978,1,3.08
2,1,2015-01-01 00:34:00,2015-01-01 00:38:00,-73.843010,40.719055,-73.846581,40.711567,1,0.90
3,2,2015-01-01 00:34:00,2015-01-01 00:38:00,-73.860825,40.757793,-73.854042,40.749821,1,0.85
4,2,2015-01-01 00:34:00,2015-01-01 01:09:00,-73.945183,40.783321,-73.989624,40.765450,1,4.91


#### 4. Now use SQL Queries to compute the average speed of each trips

In [12]:
# conn = sqlite3.connect('TestDB1.db')
# c = conn.cursor()

# dataset.to_sql('Green', conn, if_exists='replace', index = False)
 
# c.execute('''  
# SELECT COUNT(*) FROM Green
#           ''')

# print(c.fetchall())

In [13]:
from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lon1, lat2, lon2): 
    
#     start = time.time()
    
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 

    c = (3956 * 2 * asin(sqrt(sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2)))

#     end = time.time()
#     print(end-start)
    
    return c
     
    # Radius of earth in kilometers. Use 3956 for miles 
    # calculate the result 
    # (c * r) 

def time_difference(time1, time2):
    time_diff = (time2 - time1)
    time_seconds = abs(time_diff.total_seconds())
    
    return time_seconds / 60.0

print(columns)

Index(['VendorID', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime',
       'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude',
       'Dropoff_latitude', 'Passenger_count', 'Trip_distance'],
      dtype='object')


In [14]:
dataset = dataset.sort_values(by=['lpep_pickup_datetime', 'Lpep_dropoff_datetime'])
# dataset = dataset.reset_index()

In [15]:
dataset['Average Speed (MPH)'] = dataset.apply(lambda row : row.Trip_distance / (time_difference(row.lpep_pickup_datetime, row.Lpep_dropoff_datetime) / 60.0), axis = 1) 

In [16]:
dataset = dataset.reset_index(drop=True)

In [18]:
dataset

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Average Speed (MPH)
0,2,2015-01-01 00:00:00,2015-01-01 00:03:00,-73.918648,40.759354,-73.912155,40.767422,1,0.60,12.000000
1,2,2015-01-01 00:00:00,2015-01-01 00:03:00,-73.892464,40.746998,-73.911209,40.744617,1,0.99,19.800000
2,2,2015-01-01 00:00:00,2015-01-01 00:04:00,-73.891365,40.746635,-73.896118,40.738361,1,0.87,13.050000
3,1,2015-01-01 00:00:00,2015-01-01 00:04:00,-73.807426,40.700104,-73.806000,40.693253,1,0.60,9.000000
4,1,2015-01-01 00:00:00,2015-01-01 00:05:00,-73.918846,40.743019,-73.901253,40.745758,1,1.00,12.000000
...,...,...,...,...,...,...,...,...,...,...
5638465,2,2015-12-01 23:59:00,2015-01-13 00:14:00,-73.957588,40.671902,-73.955826,40.719593,1,4.00,0.000516
5638466,2,2015-12-01 23:59:00,2015-01-13 00:14:00,-73.953896,40.663887,-73.916283,40.686119,2,3.33,0.000430
5638467,2,2015-12-01 23:59:00,2015-01-13 00:15:00,-73.954567,40.718754,-73.978569,40.665421,1,7.25,0.000935
5638468,1,2015-12-01 23:59:00,2015-01-13 00:24:00,-73.988892,40.692158,-73.988770,40.777615,1,7.20,0.000929


In [35]:
def mergeable_check(trip1, trip2, delay):
    
    avg_speed = (trip1['Average Speed (MPH)'] + trip2['Average Speed (MPH)'])/2
    
    Do1o2 = distance(trip1['Pickup_latitude'], trip1['Pickup_longitude'], trip2['Pickup_latitude'], trip2['Pickup_longitude'])
    Dd1d2 = distance(trip1['Dropoff_latitude'], trip1['Dropoff_longitude'], trip2['Dropoff_latitude'], trip2['Dropoff_longitude'])

    Do2d1 = distance(trip2['Pickup_latitude'], trip2['Pickup_longitude'], trip1['Dropoff_latitude'], trip1['Dropoff_longitude'])
    Do2d2 = distance(trip2['Pickup_latitude'], trip2['Pickup_longitude'], trip2['Dropoff_latitude'], trip2['Dropoff_longitude'])
    
    To1o2 = Do1o2/avg_speed
    Td1d2 = Dd1d2/avg_speed
    
    To2d1 = Do2d1/avg_speed
    To2d2 = Do2d2/avg_speed
    
#     print(max([Do2d1, Do2d2]) + Do1o2 + Dd1d2, "|", trip1['Trip_distance'] + trip2['Trip_distance'])
    
#     sequence1 = False
#     sequence2 = False
    
#     sequence1Time = Trip1['lpep_pickup_datetime']
#     sequence2Time = Trip2['lpep_pickup_datetime']
    
    dist = max([Do2d1, Do2d2]) + Do1o2 + Dd1d2
    if (dist < trip1['Trip_distance'] + trip2['Trip_distance']):
#         print('True', dist, trip1['Trip_distance'] + trip2['Trip_distance'])
        return (True, dist)
    else:
#         print('False', dist, trip1['Trip_distance'] + trip2['Trip_distance'])
        return (False, 0)
    
#     if (sequence1 or sequence2): return True

In [38]:
trips_processed = 0
mergeable_trips = 0

def shared_trips_eval(dataset, delay, length):    
    
    global trips_processed 
    global mergeable_trips
    
    for index1 in range(length):
        index2 = index1 + 1
        rides = []
        Trip1 = dataset.iloc[index1]
        
        trips_processed += 1
        
        while (index2 < length):
            Trip2 = dataset.iloc[index2]
            if (Trip1['Passenger_count'] + Trip2['Passenger_count'] <= 3):
                if (time_difference(Trip1['lpep_pickup_datetime'], Trip2['lpep_pickup_datetime']) > delay or time_difference(Trip1['Lpep_dropoff_datetime'], Trip2['Lpep_dropoff_datetime']) > delay):
                    break
            
            mergeable, dist = mergeable_check(Trip1, Trip2, delay)
            if (mergeable):
                rides.append(index2)
                mergeable_trips += 1
            index2 = index2 + 1
        print(index1, rides)
#         print('')
#         if (index1 % 200 == 0):
#             print(index1)
    print("Done")

start = time.time()
shared_trips_eval(dataset[:500], 5.0, len(dataset[:500]))
# end = time.time()
# print('')
# print('Trips processed: ', trips_processed, 'Time Taken: ' ,end-start)

0 []
1 []
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 [12]
12 []
13 []
14 [15]
15 []
16 []
17 [21]
18 [22]
19 [20, 24]
20 [24]
21 [22, 23]
22 [23, 25]
23 [25]
24 []
25 []
26 []
27 []
28 []
29 []
30 []
31 []
32 [36, 42]
33 []
34 []
35 []
36 [42, 48]
37 []
38 [40, 41, 46]
39 [40]
40 [46]
41 [45]
42 [48]
43 [47, 49, 52]
44 []
45 []
46 []
47 [49, 53]
48 []
49 [52, 53]
50 []
51 [57]
52 [53]
53 [58]
54 [57]
55 []
56 [57, 60]
57 [60]
58 [61]
59 []
60 [62]
61 []
62 [63]
63 []
64 []
65 []
66 []
67 [71]
68 []
69 []
70 []
71 [75, 87]
72 []
73 []
74 []
75 [87]
76 []
77 []
78 []
79 []
80 []
81 [83]
82 [83]
83 []
84 [92, 94]
85 []
86 []
87 [94, 95]
88 []
89 [91]
90 [92, 94, 95, 96, 98]
91 []
92 [94, 95]
93 [99]
94 [95, 98]
95 [98]
96 [98, 102]
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 [106, 107]
106 [107]
107 [108]
108 []
109 []
110 []
111 []
112 []
113 [125]
114 []
115 []
116 [121]
117 [129]
118 []
119 [125]
120 []
121 []
122 []
123 []
124 [129]
125 []
126 [131]
127 []
128 []
12

In [39]:
end = time.time()
print('')
print('Trips processed: ', trips_processed, ', MergeAble Trips: ', mergeable_trips, ', Time Taken: ' ,end-start)
print('Last Trip Processed: ')
print(dataset.iloc[trips_processed])


Trips processed:  500 , MergeAble Trips:  355 , Time Taken:  9.243360996246338
Last Trip Processed:  VendorID                                   2
lpep_pickup_datetime     2015-01-01 00:09:00
Lpep_dropoff_datetime    2015-01-01 00:20:00
Pickup_longitude                    -73.9535
Pickup_latitude                      40.8037
Dropoff_longitude                   -73.9505
Dropoff_latitude                     40.8248
Passenger_count                            1
Trip_distance                           1.85
Average Speed (MPH)                  10.0909
Name: 500, dtype: object
